<a href="https://colab.research.google.com/github/berrygayo/Deep-learning-to-learn-from-the-founder-of-Keras/blob/master/3_5%EB%89%B4%EC%8A%A4%EA%B8%B0%EC%82%AC%EB%B6%84%EB%A5%98_%EB%8B%A4%EC%A4%91%EB%B6%84%EB%A5%98%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.5 뉴스 기사 분류 : 다중 분류 문제(multiclass classifiacation)
+ 로이터(Reuter) 뉴스를 46개의 상호 배타적인 토픽으로 분류하는 신경망 
+ 각 데이터 포인트가 정확히 하나의 범주로 분류되기 때문에 좀 더 정확히 말하면 단일 레이블 다중 분류 문제(single-label, multiclass claddification)이다.
+ 각 데이터 포인트가 여러 개의 범주(예를들어 토픽)에 속할 수 있다면 이것은 다중 레이블 다중 분류(multi-label, multiclass classification) 문제이다. 

## 3.5.1 로이터 데이터 셋 
+ 1986년에 로이터에서 공개한 짧은 뉴스 기사와 토픽의 집합 
46개의 토픽이 있으며 어떤 토픽은 다른 것에 비해 데이터가 많다. 각 토픽은 훈련 세트에 최소한 10개의 샘플을 가지고 있습니다. 
IMDB, MNIST와 마찬가지로 로이터 데이터셋은 케라스에 포함되어 있습니다. 

In [ ]:
from keras.datasets import reuters
(train_data, train_labels), (test_data, test_labels)=reuters.load_data(num_words=10000) #데이터에서 가장 자주 등장하는 단어 1만개로 제한 
# 여기에는 8982개의 훈련 샘플과 2246개의 테스트 샘플이 있다 .

In [ ]:
# 여기에는 8982개의 훈련 샘플과 2246개의 테스트 샘플이 있다 .
len(train_data)
len(test_data)

In [ ]:
# 각 샘플은 정수 리스트입니다(단어인덱스)
train_data[10]

In [ ]:
word_index=reuters.get_word_index()
reverse_word_index=dict([(value,key) for (key,value) in word_index.items()])
decoded_newswire=' '.join([reverse_word_index.get(i-3,'?') for i in train_data[0]]) #0,1,2는 '패딩','문서 시작','사전에 없음'을 위한 인덱스이므로 3을 뺍니다. 

In [ ]:
word_index

In [ ]:
reverse_word_index

In [ ]:
train_data[0]

In [ ]:
decoded_newswire

## 3.5.2 데이터 준비 

### 데이터 > 벡터로 

In [ ]:
import numpy as np 

def vectorize_sequences(sequences,dimension=10000):
    results=np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i,sequence]=1.
    return results 

x_train=vectorize_sequences(train_data)
x_test=vectorize_sequences(test_data)

In [ ]:
np.zeros((3,2))

In [ ]:
train_data[0]

In [ ]:
x_train[0]

### 레이블 > 벡터로 
+ 레이블의 리스트를 정수 텐서로 변환 
+ 원-핫 인코딩을 사용하는 것 

원-핫 인코딩이 범주형 데이터에 널ㄹ ㅣ사용되기 때문에 범주형 인코딩(categorical encoding)이라고도 부릅니다. 원-랏 인코딩에 대한 자세한 설명은 6.1절을 참고하세요. 이 경우 레이블의 원-핫 인코딩은 각 레이블의 인덱스 자리는 1이고 나머지는 모두 0인 벡터입니다. 

In [ ]:
## 원핫 인코딩 코드 
def to_one_hot(labels, dimension=46):
    results=np.zeros((len(labels),dimension))
    for i, label in enumerate(labels):
        results[i,label]=1.
    return results 

one_hot_train_labels=to_one_hot(train_labels)
one_hot_test_labels=to_one_hot(test_labels)

In [ ]:
train_labels[0]

In [ ]:
one_hot_train_labels[0]

keras에는 이를 위한 내장 함수가 있다.

In [ ]:
from keras.utils.np_utils import to_categorical

one_hot_train_labels=to_categorical(train_labels)
one_hot_test_labels=to_categorical(test_labels)

## 3.5.3 모델 구성 
이전 영화 리뷰 분류 문제와 비슷해보이지만, 출력 클래스의 개수가 2개에서 46개로 늘었다. (출력 공간의 차원이 훨씬 커졌다.)
이전에 사용했던 것처럼 Dense 층을 쌓으면 각  층은 이전 층의 출력에서 제공한 정보만 사용할 수 없다. 
한 층이 분류 문제에 필요한 일부 정보를 누락하면 그다음 층에서 이를 복할 방법이 없습니다. 
각 층은 잠재적으로 정보의 병목(information bottleneck)이 될 수 있습니다.
이전 예제에서 16차원을 가진 중간층을 사용했지만 16차원 공간은 46개의 클래스를 구분하기에 너무 제약되는것이 많을 것 같습니다.
이렇게 규모가 작은 층은 유용한 정보를 완전히 잃게 되는 정보의 병목 지점처럼 동작할 수 있습니다. 
이런 이유로 좀 더 규모가 큰 층을 사용하겠습니다 > 64개의 유닛 사용 


> 정보의 병목이 무엇인가? information bottleneck

좁은 병 입구에 대량의 물을 집어 넣음 

 전체 시스템의 성능이나 용량이 하나의 구성 요소로 인해 제한을 받는 현상을 말한다.  > 시스템에서의 병목현상 


In [ ]:
from keras import models 
from keras import layers 

model=models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(46,activation='softmax'))

# softmax 활성화 함수 > 각 입력 샘플마다 46개의 출력 클래스에 대한 확률 분포를 출력한다. 
# 즉 46차원의 출력 벡터를 만들어 output[i]는 어떤 샘플이 클래스 i에 속할 확률이다. 46개의 값을 모두 더하면 1이 된다. 

# 이진분류 > sigmoid, 다중분류 > softmax 

In [ ]:
# 이런 문제에 사용할 최선의 손실 함수는 categorical_crossentropy입니다. 이 함수는 두 확률 분포 사이의 거리르 ㄹ측정합니다. 
# 여기서는 네트워크가 출력한 확률 분포와 진짜 레이블의 분포 사이의 거리 
# 두 분포 사이의 거리를 최소화하면 진짜 레이블에 가능한 가까운 출력을 내도록 모델을 훈련한다
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## 3.5.4 훈련 검증 
훈련 데이터에서 1000개의 샘플을 따로 떼어서 검증 세트로 사용하겠습니다 

In [ ]:
x_val=x_train[:1000]
partial_x_train=x_train[1000:]

y_val=one_hot_train_labels[:1000]
partial_y_train=one_hot_train_labels[1000:]

In [ ]:
# 20번의 에포크로 모델을 훈련시킨다.
history=model.fit(partial_x_train, 
                  partial_y_train,
                  epochs=20,
                  batch_size=512,
                  validation_data=(x_val,y_val))

In [ ]:
# 훈련과 검증 손실 그리기 
import matplotlib.pyplot as plt 

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(1,len(loss)+1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf() #그래프를 초기화 

acc=history.history['accuracy']
val_acc=history.history['val_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


### 이 모델은 아홉 번째 에포트 이후에 과대적합이 시작됩니다. 아홉 번째 에포크로 새로운 모델을 훈련하고 테스트 세트에서 평가하겠습니다. 


In [ ]:
from keras import models 
from keras import layers 

model=models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(46,activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train, 
                  partial_y_train,
                  epochs=9,#9번 학습으로 변경 
                  batch_size=512,
                  validation_data=(x_val,y_val))

results=model.evaluate(x_test,one_hot_test_labels)

In [ ]:
# 최종 결과는 다음과 같다 
results

### 대략 78%의 정확도를 달성. 균형 잡힌 이진 분류 문제에서 완전히 무작위로 분류하면 50%의 정확도를 달성합니다. 이 문제는 불균형한 데이터셋을 사용하므로 무작위로 분류하면 18% 정도를 달성합니다. 여기에 비하면 이 결과는 꽤 좋은 편입니다. 

In [ ]:
import copy 
test_labels_copy=copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
hits_array=np.array(test_labels)==np.array(test_labels_copy)
float(np.sum(hits_array))/len(test_labels)

In [ ]:
test_labels # 진짜 

In [ ]:
test_labels_copy #랜덤으로 섞음 

# 3.5.5 새로운 데이터에 대해 예측하기 
모델 객체의 predict 메서드는 46개의 토픽에 대한 확률 분포를 반환합니다.
테스트 데이터 전체에 대한 토픽을 예측해 보겠습니다. 

In [ ]:
predictions=model.predict(x_test)


In [ ]:
predictions[0].shape # 각 항목은 길이가 46인 벡터이다. 

In [ ]:
np.sum(predictions[0]) #이 벡터의 원소 합은 1이다 

In [ ]:
np.argmax(predictions[0]) # 가장 높은 확률이 예측 클라스가 됨 > 4 

# 3.5.6 레이블과 손실을 다루는 다른 방법
앞서 언급한 것처럼 레이블을 인코딩하는 다른 방법은 다음과 같이 정수 텐서로 변환하는 것이다.

In [ ]:
y_train=np.array(train_labels)
y_test=np.array(test_labels)

이 방식을 사용하려면 손실 함수 하나만 바꾸면 됩니다. 앞서 사용한 categorical_crossentropy는 레이블이 범주형 인코딩되어 있을 것이라고 기대합니다. 정수 레이블을 사용할 때는 sparse_categorical_crossentripy를 사용해야합니다. 

In [ ]:
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

# 이 손실 함수는 인터페이스만 다를 뿐이고 수학적으로는 categorical_crossentropy 와 동일합니다. 

# 3.5.7 충분히 큰 중간층을 두어야 하는 이유 
앞서 언급한 것처럼 마지막 출력이 46차원이기 때문에 중간층의 히든 유닛이 46개보다 많이 적어서는 안 됩니다. 46차원보다 훨씬 작은 중간층(예를들어 4차원)을 두면 정보의 병목이 어떻게 나타나는지 확인해보겠습니다. 

In [ ]:
model=models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(4,activation='relu'))
model.add(layers.Dense(46,activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(partial_x_train, 
                  partial_y_train,
                  epochs=20,
                  batch_size=512,
                  validation_data=(x_val,y_val))

검증 정확도의 최고 값은 약 69%로 10% 정도 감소했습니다. 
이런 손실의 원인 대부분은 많은 정보(클래스 46개의 분할 초평면을 복원하기에 충분한 정보)를 중간층의 저차원 표현 공간으로 압축하려고 했기 때문입니다.
이 네트워크는 필요한 정보 대부분을 4차원 표현 안에 구겨 넣었지만 전부는 넣지 못했습니다.


## 3.5.9 정리 
+ N개의 클래스로 데이터 포인트를 분류하려면 네트워크의 마지막 Dense 층의 크기는 N이어야 합니다. 
+ 단일 레이블, 다중 분류 문제에서는 N개의 클래스에 대한 확률 분포를 출력하기 위해 softmax 활성화 함수를 사용해야 합니다. 
+ 이런 문제에는 항상 범주형 크로스엔트로피를 사용해야 합니다. 이 함수는 모델이 출력한 확률 분포와 타깃 분포 사이의 거리를 최소화합니다. 
+ 다중 분류에서 레이블을 다루는 두 가지 방법이 있습니다 
  > 레이블을 범주형 인코딩(또는 원-핫 인코딩)으로 인코딩하고 categorical_crossentropy 손실 함수를 사용합니다.
  
  > 레이블을 정수로 인코딩하고 spaese_categorical_crossentropy손실 함수를 사용합니다. 
+많은 수의 범주를 분류할 때 중간층의 크기가 너무 작아 네트워크에 정보의 병목이 생기지 않도록 해야 합니다. 